# Python Connectivity Demo

### Demo setup

1. Have a Python 3.7+ environment ready
2. Open this file in a Jupyter-friendly editor (e.g. VSCode)
3. Install the following Python packages using `pip`:

        pip install -r requirements.txt

4. In the following Python block - replace the string placeholders the actual connection properties, and execute

In [ ]:
# Replace the string placeholders with your actual connection info!
HOST     = 'host'       # e.g. "yourenv.isqream.com"
USERNAME = 'username'   # e.g. "ben"
PASSWORD = 'password'   # e.g. "Aa123456"
DATABASE = 'database'   # e.g. "master"

5. Continue executing the rest of this guide in Jupyter

### Intro to Jupyter Notebook

Jupyter is a multi-language environment which allows you to combine code, text and visuals. Interactive execution makes it great for interactive exploration, as well as for demos and tutorials.

Go ahead and execute the following Python cell:

In [ ]:
x = 1
y = 2
print(f'x + y = {x+y}')

The rest of this Notebook will demonstrate the various ways you can connect to SQream Blue from a Python environment.

### Basic connectivity with Python DB-API

The quickest way to connect to SQream is by using SQream's native DB-API Python connector - pysqream.

In [ ]:
import pysqream_blue as dbapi
con = dbapi.connect(host=HOST, database=DATABASE, username=USERNAME, password=PASSWORD)

Statements can be executed by creating a `cursor` object:

In [ ]:
cur = con.cursor()
cur.execute("SELECT 1 as x, 2 as y")
results = cur.fetchall()
print(results)

Result data is represented as a list of lists, meaning it can be easily unpacked and used in subsequent Python code:

In [ ]:
first_row = results[0]
x = first_row[0]
y = first_row[1]
print(f'x + y = {x+y}')

Lastly - let's close the cursor and the connection objects

In [ ]:
cur.close()
con.close()

### SQLAlchemy - better integration with the Python data ecosystem

In [ ]:
import sqlalchemy as sa
sqlalchemy_conn_string = f'sqream_blue://{USERNAME}:{PASSWORD}@{HOST}/{DATABASE}'
engine = sa.create_engine(sqlalchemy_conn_string)

Queries are executed using a `connection` object, and results are returned as a list of tuples:

In [ ]:
conn = engine.connect()
results = conn.execute('select 1 as x, 2 as y').fetchall()
print(results)

The main advantage of using SQLAlchemy is that it integrates nicely with other popular Python libraries. Let's see how we can use it to query SQream directly from Pandas:

In [ ]:
import pandas as pd
df = pd.read_sql("select schema_id,schema_name from sqream_catalog.schemas", con = conn)
df.info()

The DataFrame is the standard in-memory columnar representation of tabular data in Python, and there's LOT of useful stuff you can do with it. In addition, it can be beautifully displayed in Jupyter:

In [ ]:
display(df)

Before we continue to the next step, we should remember to close the SQLAlchemy connection:

In [ ]:
conn.close()
engine.dispose()

### Jupyter magic integration with IPython-sql/jupysql

Up until now, we've been using regular Python code, meaning that you can use these samples as part of regular Python scripts.

In this last section we'll use a Jupyter-only feature called _Jupyter Magic_, which will allow us to execute SQL in an almost native way, directly from the Notebook cells. This is made possibly by the `jupysql` package, which provides the `%sql` macro.

In the following cells, we use several Jupyter _magic commands_ in order to set up an SQLAlchemy connection and link it to the `%sql` command.

In [ ]:
%load_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.displaycon=False
%config SqlMagic.autopandas=True
%env DATABASE_URL $sqlalchemy_conn_string

We can now use the `%sql` magic command to send one-line SQL queries to SQream, creating a CLI-like experience:

In [ ]:
%sql create or replace table t(x int not null);
%sql insert into t values (1),(2),(3),(4);
%sql select * from t

The `%sql` command loads the query results to a Pandas DataFrame. This DataFrame can be saved as a Python variable, which makes it easy to combine the SQL results with regular Python code:

In [ ]:
import matplotlib.pyplot as plt
df = %sql select x,power(x,2) as sqr from t
plt.bar(df["x"], df["sqr"])
plt.title("x vs sqr(x)")
plt.show()

For long queries, using the one-line `%sql` command can be less than ideal. This is where the entire-cell `%%sql` magic comes in:

In [ ]:
%%sql
select
  x,
  power(x,2) as sqr
from t

Note that `%%sql` can only appear in the beginning of the Jupyter cell.